In [1]:
import pandas as pd
import numpy as np
from glob import glob


In [2]:


file_lst = glob("raw/*.csv")
file_lst.remove("raw/UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv")
features=[]
with open("raw/UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv", "r") as f:
    features = f.read().split(",")
features

['pkSeqID',
 'stime',
 'flgs',
 'proto',
 'saddr',
 'sport',
 'daddr',
 'dport',
 'pkts',
 'bytes',
 'state',
 'ltime',
 'seq',
 'dur',
 'mean',
 'stddev',
 'smac',
 'dmac',
 'sum',
 'min',
 'max',
 'soui',
 'doui',
 'sco',
 'dco',
 'spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'rate',
 'srate',
 'drate',
 'attack',
 'category',
 'subcategory ']

In [3]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [4]:

n_normal = 0
n_attack = 0
req_ft=['pkSeqID','stime','pkts','bytes','ltime','seq','dur','mean','stddev','sum','min','max','spkts','dpkts','sbytes','dbytes','rate','srate','drate', 'attack']

print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos', 'scanning'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    
    print(len(r))    
    
    rng = np.random.default_rng()
    r.add(rng.choice(df.to_numpy(), size=max(0,int(size*0.03)), replace=False))
    del df
    


20


KeyError: 'type'

In [ ]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 16680497


In [ ]:
df = r.getDf()
df.to_csv("processed/imbalanced_dataset.csv", index=False)

In [ ]:


df = pd.read_csv("processed/imbalanced_dataset.csv")
print(df[df.isnull().any(axis=1)].shape[0])
N=df.shape[0]
train_size=int((N*3)/4)
print(f"N: {N}, TS:{train_size}")
test_df = df.loc[train_size:N].sample(frac=1)
df = df.loc[0:train_size]

0
N: 523066, TS:392299


In [ ]:
test_df.to_csv('processed/test_data.csv', index=False)

In [5]:
# from imblearn.over_sampling import ADASYN
# Y = df['label'].to_numpy().astype(np.uint8)
# X = df.drop('label', axis=1).to_numpy()
# df[df['label'].isnull()].head()
# print(Y[0:100])



In [6]:
# from imblearn.over_sampling import ADASYN
# X_r, Y_r = ADASYN().fit_resample(X, Y)
# res=np.empty((X_r.shape[0], df.shape[1]))
# res[:, 0:-1] = X_r
# res[:, -1] = Y_r
# pd.DataFrame(res, columns = df.columns).to_csv("processed/balanced_dataset.csv", index=False)

NameError: name 'X' is not defined

In [18]:
np.bincount(Y_r)

array([381013, 379951])

In [19]:
np.random.shuffle(res)


In [20]:
pd.DataFrame(res, columns = df.columns).to_csv("processed/train_data.csv", index=False)

In [21]:
pd.DataFrame(res).tail(15)

,0,1,2,3,4,5,6
760949,0.000003,0.000000,0.0,0.0,2.000000,1.0,1.0
760950,60.460211,534.000000,1727.0,0.0,7.000000,16.0,1.0
760951,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0
760952,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0
760953,0.000000,0.000000,0.0,0.0,1.000000,0.0,1.0
760954,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0
760955,0.002411,39.000000,39.0,0.0,1.000000,1.0,1.0
760956,0.000000,0.000000,0.0,0.0,1.000000,0.0,1.0
760957,0.000000,0.000000,0.0,0.0,1.000000,0.0,1.0
760958,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0
